In [ ]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

In [ ]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("LoadingData").getOrCreate()

In [ ]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://spot-i-fy-bucket.s3.us-east-2.amazonaws.com/tracks.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("tracks.csv"), sep=",", header=True)

# Show DataFrame
df.show()

In [ ]:
# Print our schema
df.printSchema()

In [ ]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://spot-i-fy-bucket.s3.us-east-2.amazonaws.com/artists.csv"
spark.sparkContext.addFile(url)
a_df = spark.read.csv(SparkFiles.get("artists.csv"), sep=",", header=True)

# Show DataFrame
a_df.show()

In [ ]:
# Import dependencies for Machine Learning
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import fcluster, linkage, dendrogram
import pandas as pd
import numpy as np
from pathlib import Path
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.cluster import KMeans

In [ ]:
# Convert Spark DataFrame to Pandas DataFrame
artists_df = pd.DataFrame(a_df)

In [ ]:
# Convert Spark DataFrame to Pandas DataFrame
track_df = pd.DataFrame(df)

In [ ]:
# Merge the two dataframes
# How to merge when columns have different name
music_df = track_df.merge(artists_df, left=['artists', 'name'])
music_df.head()

In [ ]:
# Exploratory Data to understand how we need to preprocess
# Explore popularity range - over 75? over 50?
music_df.dtypes

In [ ]:
# Drop columns that won't be useful to clustering by genre
# Is it okay to not merge for Machine Learning, but just for visualizations? The artists df gives us the answer
# Drop id, name, popularity, duration_ms, explicit, artists, id_artists, release_date, 
spotify_df = music_df.drop(columns=['id', 'name', 'popularity', 'duration_ms', 'explicit', 'artists', ''])

In [ ]:
# Preprocessing
# US only
# Popularity? What's a good cut off?

In [ ]:
# Feature selection: 

In [ ]:
# Use Hierarchical clustering
# Read data from DB using PySpark
# Load in Track.csv from S3 into a DataFrame

# Normalize data
from sklearn.preprocessing import normalize
data_scaled = normalize(data)
data_scaled = pd.DataFrame(data_scaled, columns=data.columns)
data_scaled.head()

In [ ]:

# Create hierarchal clustering array using n_clusters determined from dendrogram analysis
from sklearn.cluster import AgglomerativeClustering
cluster = AgglomerativeClustering(n_clusters=X, affinity='euclidean', linkage='ward')  
cluster.fit_predict(data_scaled)

In [ ]:
# Plot scatter plot of clusters

In [ ]:
# Does using PCA help? Some clusters are opposite/inverse of others, so it might hurt

In [ ]:
# Go through a few models. 3 maybe? Why do I think the best performing model is best performing?

# Notes from class! Is this data set actually labeled? Maybe I want to see if this can predict genre?
# How would I design a neural network? Use that loop to pick the number of features?
# How accurate/good is each model?
# Were we able to use a simple model?
# Do we get similar results each time? 

In [ ]:
# I'll do k means clustering and round it out with a neural network
# Question is, can we predict the genre lable?
# Talk about stuff over time for the tableau bit

In [ ]:
#K Means Clustering
#Begin with elbow graph to determine how many clusters we might want
inertia = []
k = list(range(1, 11))
# Calculate the inertia for the range of K values
for i in k:
   km = KMeans(n_clusters=i, random_state=0)
   km.fit(df_shopping)
   inertia.append(km.inertia_)
#Create elbow curve in hvPlot
elbow_data = {"k":k, "inertia":inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x-"k", y="inertia", xticks=k, title="Elbow Curve")

In [ ]:
# Set X 
X = spotify_df
X_scaled = StandardScaler().fit_transform(X)

In [ ]:
# Create K Means Model with X clusters (from elbow plot)
model = KMeans(n_clusters=3, random_state=42).fit(X_scaled)

In [ ]:
# Calculate predicted values.
y_pred = model.predict(X_scaled)

In [ ]:
# Add predicted values onto the original dataframe
df_y = pd.DataFrame(y_pred, columns=['Cluster'])
combined = df.join(df_y, how='inner')
combined.head()

In [ ]:
# Plot clusters
# Create a scatterplot of df_iris
#df_iris.hvplot.scatter(x="SOME COLUMN", y="SOME COLUMN", by="SOME COLUMN")
fig.update_layout(legend=dict(x=0,y=1))
fig.show()

In [ ]:
#Neural Network
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import tensorflow as tf

In [ ]:
# Use the preprocessed data from the previous 
# Wait can I even use a normal neural network? 
# https://en.wikipedia.org/wiki/Self-organizing_map